In [5]:
import os
import networkx as nx
from community import community_louvain
from networkx.algorithms.community import girvan_newman
from sklearn.metrics import normalized_mutual_info_score, f1_score
import itertools

In [9]:
def load_facebook_ego_graph(base_dir, node_id):
    edges_file = os.path.join(base_dir, f"{node_id}.edges")
    circles_file = os.path.join(base_dir, f"{node_id}.circles")

    G = nx.Graph()
    with open(edges_file, 'r') as ef:
        for line in ef:
            u, v = line.strip().split()
            G.add_edge(u, v)

    ground_truth_communities = []
    if os.path.exists(circles_file):
        with open(circles_file, 'r') as cf:
            for line in cf:
                parts = line.strip().split()
                members = parts[1:]
                members_in_graph = [m for m in members if m in G.nodes()]
                if len(members_in_graph) > 0:
                    ground_truth_communities.append(set(members_in_graph))
    else:
        ground_truth_communities = []

    return G, ground_truth_communities

In [10]:
def detect_communities_louvain(G):
    partition = community_louvain.best_partition(G)
    communities = {}
    for node, com_id in partition.items():
        communities.setdefault(com_id, []).append(node)
    return [set(c) for c in communities.values()]

def detect_communities_girvan_newman(G, top_level=1):
    comp = girvan_newman(G)
    for level in range(top_level):
        communities = next(comp)
    return [set(c) for c in communities]

def f1_for_sets(true_set, pred_set):
    inter = len(true_set.intersection(pred_set))
    precision = inter / len(pred_set) if len(pred_set) > 0 else 0.0
    recall = inter / len(true_set) if len(true_set) > 0 else 0.0
    if (precision + recall) > 0:
        return 2 * (precision * recall) / (precision + recall)
    else:
        return 0.0


In [7]:
def detect_communities_louvain(G):
    partition = community_louvain.best_partition(G)
    communities = {}
    for node, com_id in partition.items():
        communities.setdefault(com_id, []).append(node)
    return [set(c) for c in communities.values()]

def detect_communities_girvan_newman(G, top_level=1):
    comp = girvan_newman(G)
    for level in range(top_level):
        communities = next(comp)
    return [set(c) for c in communities]


In [11]:
def evaluate_communities(detected_communities, ground_truth_communities):
    if not ground_truth_communities:
        return 0.0, 0.0

    all_nodes = set().union(*ground_truth_communities).union(*detected_communities)
    all_nodes = list(all_nodes)
    node_index = {n: i for i,n in enumerate(all_nodes)}

    def communities_to_labels(communities):
        labels = [-1]*len(all_nodes)
        max_cid = 0
        for cid, com in enumerate(communities):
            for node in com:
                if labels[node_index[node]] == -1:
                    labels[node_index[node]] = cid
                    if cid > max_cid:
                        max_cid = cid
        for i, l in enumerate(labels):
            if l == -1:
                max_cid += 1
                labels[i] = max_cid
        return labels

    gt_labels = communities_to_labels(ground_truth_communities)
    det_labels = communities_to_labels(detected_communities)
    nmi = normalized_mutual_info_score(gt_labels, det_labels)

    f1_scores = []
    for gt_com in ground_truth_communities:
        best_f1 = 0.0
        for dt_com in detected_communities:
            score = f1_for_sets(gt_com, dt_com)
            if score > best_f1:
                best_f1 = score
        f1_scores.append(best_f1)
    avg_f1 = sum(f1_scores)/len(f1_scores) if f1_scores else 0.0

    return nmi, avg_f1

In [12]:
if __name__ == "__main__":
    base_dir = "facebook"
    node_ids = ["0","107","348","414","686","698","1684","1912","3437","3980"]

    print("node_id\tLouvain_NMI\tLouvain_F1\tGN_NMI\tGN_F1")
    for nid in node_ids:
        G, ground_truth = load_facebook_ego_graph(base_dir, nid)

        louvain_coms = detect_communities_louvain(G)
        louvain_nmi, louvain_f1 = evaluate_communities(louvain_coms, ground_truth)

        gn_coms = detect_communities_girvan_newman(G, top_level=1)
        gn_nmi, gn_f1 = evaluate_communities(gn_coms, ground_truth)

        print(f"{nid}\t{louvain_nmi:.4f}\t{louvain_f1:.4f}\t{gn_nmi:.4f}\t{gn_f1:.4f}")

node_id	Louvain_NMI	Louvain_F1	GN_NMI	GN_F1
0	0.4341	0.2595	0.2688	0.1988
107	0.4368	0.3274	0.0149	0.0917
348	0.2702	0.4672	0.2869	0.3276
414	0.6581	0.5362	0.3957	0.3912
686	0.4667	0.4630	0.0236	0.3243
698	0.7003	0.5110	0.5440	0.3884
1684	0.6786	0.4060	0.1471	0.2000
1912	0.4940	0.1852	0.0414	0.0998
3437	0.5087	0.1111	0.0560	0.0306
3980	0.6874	0.4587	0.4384	0.3295


In [1]:
import os
import networkx as nx
from community import community_louvain
from networkx.algorithms.community import girvan_newman
from sklearn.metrics import normalized_mutual_info_score
import statistics

def load_facebook_ego_graph(base_dir, node_id):
    edges_file = os.path.join(base_dir, f"{node_id}.edges")
    circles_file = os.path.join(base_dir, f"{node_id}.circles")

    G = nx.Graph()
    with open(edges_file, 'r') as ef:
        for line in ef:
            u, v = line.strip().split()
            G.add_edge(u, v)

    ground_truth_communities = []
    if os.path.exists(circles_file):
        with open(circles_file, 'r') as cf:
            for line in cf:
                parts = line.strip().split()
                members = parts[1:]
                members_in_graph = [m for m in members if m in G.nodes()]
                if len(members_in_graph) > 0:
                    ground_truth_communities.append(set(members_in_graph))
    else:
        ground_truth_communities = []

    return G, ground_truth_communities

def detect_communities_louvain(G):
    partition = community_louvain.best_partition(G)
    communities = {}
    for node, com_id in partition.items():
        communities.setdefault(com_id, []).append(node)
    return [set(c) for c in communities.values()]

def detect_communities_girvan_newman(G, top_level=1):
    comp = girvan_newman(G)
    for level in range(top_level):
        communities = next(comp)
    return [set(c) for c in communities]

def f1_for_sets(true_set, pred_set):
    inter = len(true_set.intersection(pred_set))
    precision = inter / len(pred_set) if len(pred_set) > 0 else 0.0
    recall = inter / len(true_set) if len(true_set) > 0 else 0.0
    if (precision + recall) > 0:
        return 2 * (precision * recall) / (precision + recall)
    else:
        return 0.0

def evaluate_communities(detected_communities, ground_truth_communities):
    if not ground_truth_communities:
        return 0.0, 0.0
    all_nodes = set().union(*ground_truth_communities).union(*detected_communities)
    all_nodes = list(all_nodes)
    node_index = {n: i for i,n in enumerate(all_nodes)}

    def communities_to_labels(communities):
        labels = [-1]*len(all_nodes)
        max_cid = 0
        for cid, com in enumerate(communities):
            for node in com:
                if labels[node_index[node]] == -1:
                    labels[node_index[node]] = cid
                    if cid > max_cid:
                        max_cid = cid
        for i, l in enumerate(labels):
            if l == -1:
                max_cid += 1
                labels[i] = max_cid
        return labels

    gt_labels = communities_to_labels(ground_truth_communities)
    det_labels = communities_to_labels(detected_communities)
    nmi = normalized_mutual_info_score(gt_labels, det_labels)

    # 计算F1
    f1_scores = []
    for gt_com in ground_truth_communities:
        best_f1 = 0.0
        for dt_com in detected_communities:
            score = f1_for_sets(gt_com, dt_com)
            if score > best_f1:
                best_f1 = score
        f1_scores.append(best_f1)
    avg_f1 = sum(f1_scores)/len(f1_scores) if f1_scores else 0.0

    return nmi, avg_f1



def compute_mS_cS(G, S):

    S = set(S)
    mS = 0
    cS = 0
    for u in S:
        for v in G[u]:
            if v in S:
                if u < v:
                    mS += 1
            else:
                cS += 1
    return mS, cS

def compute_scoring_functions(G, communities):
    """

    nS = |S|
    mS = #edges inside S
    cS = #edges leaving S
    d(u) = degree of node u
    """
    n = G.number_of_nodes()
    m = G.number_of_edges()

    degrees = {node: G.degree(node) for node in G.nodes()}
    all_degs = sorted(degrees.values())
    dm = statistics.median(all_degs) if all_degs else 0.0

    results = []
    for S in communities:
        nS = len(S)
        if nS <= 1:

            results.append({"Internal density":0,"Edges inside":0,"Average degree":0,"FOMD":0,"TPR":0,
                            "Expansion":0,"Cut Ratio":0,"Conductance":0,"Normalized Cut":0,"Max-ODF":0,
                            "Avg-ODF":0,"Flake-ODF":0,"Modularity":0})
            continue
        mS, cS = compute_mS_cS(G, S)


        if nS*(nS-1)/2 > 0:
            internal_density = mS/(nS*(nS-1)/2)
        else:
            internal_density = 0

        edges_inside = mS

        avg_degree = (2*mS/nS) if nS>0 else 0

        internal_degs = []
        for u in S:
            internal_deg_u = len([v for v in G[u] if v in S])
            internal_degs.append(internal_deg_u)
        count_over_dm = sum(1 for d_iu in internal_degs if d_iu > dm)
        FOMD = count_over_dm/nS

        triad_count = 0
        S_subgraph = G.subgraph(S)
        for u in S:
            neighbors_in_S = [x for x in G[u] if x in S]
            found_triangle = False
            for i in range(len(neighbors_in_S)):
                for j in range(i+1, len(neighbors_in_S)):
                    v = neighbors_in_S[i]
                    w = neighbors_in_S[j]
                    if S_subgraph.has_edge(v,w):
                        found_triangle = True
                        break
                if found_triangle:
                    break
            if found_triangle:
                triad_count += 1
        TPR = triad_count/nS

        expansion = cS/nS if nS>0 else 0

        if nS*(n-nS) > 0:
            cut_ratio = cS/(nS*(n-nS))
        else:
            cut_ratio = 0

        if (2*mS+cS) > 0:
            conductance = cS/(2*mS+cS)
        else:
            conductance = 0

        denom_2 = 2*(m - mS)+ cS
        if denom_2 > 0:
            normalized_cut = conductance + cS/denom_2
        else:
            normalized_cut = conductance

        max_odf = 0
        avg_odf_vals = []
        flake_odf_count = 0
        for u in S:
            deg_u = degrees[u]
            internal_deg_u = len([v for v in G[u] if v in S])
            external_deg_u = deg_u - internal_deg_u
            if deg_u > 0:
                out_frac = external_deg_u/deg_u
            else:
                out_frac = 0
            avg_odf_vals.append(out_frac)
            if out_frac > max_odf:
                max_odf = out_frac
            if internal_deg_u < deg_u/2:
                flake_odf_count += 1

        # Average-ODF
        avg_odf = sum(avg_odf_vals)/len(avg_odf_vals) if avg_odf_vals else 0
        # Flake-ODF = fraction of nodes with fewer than half their edges inside
        flake_odf = flake_odf_count/nS

        # Modularity: f(S) = (mS - E(mS)) / (4),:
        # Modularity = (mS/ m) - (sum(deg(u) for u in S)^2)/(4*m^2)
        sum_deg_S = sum(degrees[u] for u in S)
        expected_mS = (sum_deg_S**2)/(4*m) if m>0 else 0
        modularity = (mS - expected_mS)/4

        res = {
            "Internal density": internal_density,
            "Edges inside": edges_inside,
            "Average degree": avg_degree,
            "FOMD": FOMD,
            "TPR": TPR,
            "Expansion": expansion,
            "Cut Ratio": cut_ratio,
            "Conductance": conductance,
            "Normalized Cut": normalized_cut,
            "Max-ODF": max_odf,
            "Avg-ODF": avg_odf,
            "Flake-ODF": flake_odf,
            "Modularity": modularity
        }
        results.append(res)
    return results

if __name__ == "__main__":
    base_dir = "facebook"
    node_ids = ["0","107","348","414","686","698","1684","1912","3437","3980"]

    print("node_id\tLouvain_NMI\tLouvain_F1\tGN_NMI\tGN_F1")
    for nid in node_ids:
        G, ground_truth = load_facebook_ego_graph(base_dir, nid)
        louvain_coms = detect_communities_louvain(G)
        louvain_nmi, louvain_f1 = evaluate_communities(louvain_coms, ground_truth)

        gn_coms = detect_communities_girvan_newman(G, top_level=1)
        gn_nmi, gn_f1 = evaluate_communities(gn_coms, ground_truth)

        print(f"{nid}\t{louvain_nmi:.4f}\t{louvain_f1:.4f}\t{gn_nmi:.4f}\t{gn_f1:.4f}")

        louvain_scores = compute_scoring_functions(G, louvain_coms)
        print(f"Louvain communities scoring for node_id={nid}:")
        for i, score_dict in enumerate(louvain_scores):
            print(f" Community {i}:")
            for k,v in score_dict.items():
                print(f"   {k}: {v:.4f}")

        gn_scores = compute_scoring_functions(G, gn_coms)
        print(f"Girvan-Newman communities scoring for node_id={nid}:")
        for i, score_dict in enumerate(gn_scores):
            print(f" Community {i}:")
            for k,v in score_dict.items():
                print(f"   {k}: {v:.4f}")


node_id	Louvain_NMI	Louvain_F1	GN_NMI	GN_F1
0	0.4359	0.2652	0.2688	0.1988
Louvain communities scoring for node_id=0:
 Community 0:
   Internal density: 0.2316
   Edges inside: 907.0000
   Average degree: 20.3820
   FOMD: 0.6966
   TPR: 0.9551
   Expansion: 6.3146
   Cut Ratio: 0.0259
   Conductance: 0.2365
   Normalized Cut: 0.3850
   Max-ODF: 0.4182
   Avg-ODF: 0.1860
   Flake-ODF: 0.0000
   Modularity: 86.6801
 Community 1:
   Internal density: 0.3883
   Edges inside: 205.0000
   Average degree: 12.4242
   FOMD: 0.5758
   TPR: 0.9091
   Expansion: 2.7273
   Cut Ratio: 0.0091
   Conductance: 0.1800
   Normalized Cut: 0.1991
   Max-ODF: 0.6667
   Avg-ODF: 0.2056
   Flake-ODF: 0.0606
   Modularity: 45.0471
 Community 2:
   Internal density: 0.1595
   Edges inside: 56.0000
   Average degree: 4.1481
   FOMD: 0.0370
   TPR: 0.6667
   Expansion: 3.8148
   Cut Ratio: 0.0125
   Conductance: 0.4791
   Normalized Cut: 0.4996
   Max-ODF: 0.7705
   Avg-ODF: 0.2007
   Flake-ODF: 0.2593
   Modulari